In [1]:
# Input info
indir = ""

cloneID_f = ""
outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clonal_shifts/donors/donor0/variants_init/knn/kparam_30/clones/"
#clone_id = "0_7"
#noIn_indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clonal_shifts/variants_init/clones/results/noInput/knn/kparam_30/"
#input_indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clonal_shifts/variants_init/clones/results/inputOnly/knn/kparam_30/"
# params
#p_thresh = 0.1 








In [27]:

from glob import glob
import pandas as pd
import numpy as np
from os.path import join, exists
import os
from tqdm.notebook import tqdm

from scipy.stats import hypergeom, fisher_exact
from statsmodels.stats import multitest 

import seaborn as sns
import matplotlib.pyplot as plt

from src import clonal_shifts as cs
from icecream import ic

from mplh import cluster_help as ch

fig_utils from mplh


In [ ]:
with open(cloneID_f, "r") as f:
    cloneIDs = f.readlines()
cloneIDs = [x.strip() for x in cloneIDs]


In [3]:
#cells_meta = pd.read_csv(cells_meta_f, sep="\t")

In [ ]:
# noIn_don_indir =  join(noIn_indir,"sepDonors", f"donor{donor}")
# noInput_hyper = pd.read_csv(join(noIn_don_indir, "hypergeo_padjusted.csv"),  index_col=0)

# noIn_don_indir =  join(noIn_indir,"sepDonors", f"donor{donor}")
# noInput_hyper = pd.read_csv(join(noIn_don_indir, "hypergeo_pvals.csv"),  index_col=0)
# noIn_shuffle_df = pd.read_csv(join(noIn_don_indir, "shuffle_results_pvals.tsv"), sep="\t", index_col=0)


# input_don_indir =  join(input_indir,"sepDonors", f"donor{donor}")
# input_hyper = pd.read_csv(join(input_don_indir, "hypergeo_padjusted.csv"), index_col=0)
# input_shuffle_df = pd.read_csv(join(input_don_indir, "shuffle_results_pvals.tsv"), sep="\t", index_col=0)

# input_shuffle_df

noInput_hyper = pd.read_csv(join(indir, "noInput_hypergeo_pvals.csv"),  index_col=0)
noIn_shuffle_df = pd.read_csv(join(indir, "noInput_shuffle_results_pvals.csv"), index_col=0)

input_hyper = pd.read_csv(join(indir, "input_hypergeo_pvals.csv"), index_col=0)
input_shuffle_df = pd.read_csv(join(indir, "input_shuffle_results_pvals.csv"), index_col=0)

input_shuffle_df

In [ ]:

# hyper = pd.read_csv(join(hypergeo_in, "hypergeo_padjusted.csv"),  index_col=0)
# shuffle_df = pd.read_csv(join(hypergeo_in, "shuffle_results_pvals.tsv"), sep="\t", index_col=0)

In [ ]:
def create_pval(clone_d, hyper_key,shuffle_key, clustIDs):
    pval_df = pd.DataFrame(columns = ["hypergeo", "global_min", "clone_min", "clone_all", "global_all"],
                           index=clustIDs)

    shuff_df = clone_d[shuffle_key].pivot(index="variable", columns="method", values="value")
    shuff_df

    pval_df.loc[shuff_df.index, shuff_df.columns] = shuff_df
    pval_df.loc[clone_d[hyper_key].index, "hypergeo"] = clone_d[hyper_key]
    #pval_df.fillna(1)     
    return pval_df


def run_ind_clone(clone_id):
    curr_outdir = join(outdir, f"cloneID_{clone_id}")
    if not os.path.exists(curr_outdir):
        os.mkdir(curr_outdir)
    clone_d = {}
    clone_d["noIn_hyper"] = noInput_hyper.loc[clone_id]
    clone_d["noIn_shuffle"] = noIn_shuffle_df.loc[noIn_shuffle_df["index"]==clone_id]

    clone_d["input_hyper"] = input_hyper.loc[clone_id]
    clone_d["input_shuffle"] = input_shuffle_df.loc[input_shuffle_df["index"]==clone_id]

    clone_d['noIn_shuffle']

    clone_d['noIn_hyper']
    clustIDs = set(clone_d['input_hyper'].index).union(set(clone_d['noIn_hyper'].index))
    clustIDs = clustIDs.union(set(clone_d['noIn_shuffle']["variable"].values).union(set(clone_d['input_shuffle']["variable"].values)))
    
    noIn_pval_df = create_pval(clone_d, hyper_key="noIn_hyper",shuffle_key="noIn_shuffle", clustIDs=clustIDs)
    input_pval_df = create_pval(clone_d, hyper_key="input_hyper",shuffle_key="input_shuffle", clustIDs=clustIDs)
    pval_df = pd.merge(input_pval_df, noIn_pval_df, left_index=True, right_index=True, 
                       how="outer", suffixes=("_Input", "_Cultured"))

    ## Sort clusters by global_min, hypergeo, 
    pval_df = pval_df.fillna(1).sort_values(["global_min_Input", "hypergeo_Input", "clone_min_Input","global_min_Cultured", "hypergeo_Cultured",  "clone_min_Cultured"])
    clone_meta = pd.DataFrame(index=pval_df.columns, columns = ["condition","size"])
    clone_meta["condition"] = ["Input" if "Input" in x else "Cultured" for x in clone_meta.index]
    pval_df.to_csv(join(curr_outdir, f"cloneID_{clone_id}_hypergeo_pval.csv"))
    ch.plot_cluster(pval_df, to_col_clust=False, col_meta=clone_meta[["condition"]])
    plt.savefig(join(curr_outdir, f"cloneID_{clone_id}_hypergeo_pval.png"),bbox_inches='tight' )
    return

In [ ]:
for c_id in cloneIDs:
    run_ind_clone(c_id)

In [7]:
# clone_d = {}
# clone_d["noIn_hyper"] = noInput_hyper.loc[clone_id]
# clone_d["noIn_shuffle"] = noIn_shuffle_df.loc[noIn_shuffle_df["index"]==clone_id]

# clone_d["input_hyper"] = input_hyper.loc[clone_id]
# clone_d["input_shuffle"] = input_shuffle_df.loc[input_shuffle_df["index"]==clone_id]

# clone_d['noIn_shuffle']

# clone_d['noIn_hyper']

# clustIDs = set(clone_d['input_hyper'].index).union(set(clone_d['noIn_hyper'].index))
# clustIDs = clustIDs.union(set(clone_d['noIn_shuffle']["variable"].values).union(set(clone_d['input_shuffle']["variable"].values)))
# clustIDs

# ## Create p-val table for the clone

# noIn_pval_df = create_pval(clone_d, hyper_key="noIn_hyper",shuffle_key="noIn_shuffle")
# input_pval_df = create_pval(clone_d, hyper_key="input_hyper",shuffle_key="input_shuffle")
# pval_df = pd.merge(input_pval_df, noIn_pval_df, left_index=True, right_index=True, 
#                    how="outer", suffixes=("_Input", "_Cultured"))

# ## Sort clusters by global_min, hypergeo, 
# pval_df = pval_df.fillna(1).sort_values(["global_min_Input", "hypergeo_Input", "clone_min_Input","global_min_Cultured", "hypergeo_Cultured",  "clone_min_Cultured"])
# clone_meta = pd.DataFrame(index=pval_df.columns, columns = ["condition","size"])
# clone_meta["condition"] = ["Input" if "Input" in x else "Cultured" for x in clone_meta.index]
# pval_df

# ch.plot_cluster(pval_df, to_col_clust=False, col_meta=clone_meta[["condition"]])